### Load YOLOv8 model (Run this first)

In [ ]:
import cv2
from ultralytics import YOLO
import time
import numpy as np
from scipy.spatial import cKDTree

model = YOLO("yolov8n-seg-custom.pt")  # 載入模型

### **Corner Detect**

#### Functions and Filters define

Functions:

In [ ]:
def color_detect(masked_image):
    hsv_ranges = {
        "white": np.array([[0, 15, 200], [75, 70, 255]], dtype=np.uint8),
        "red": np.array([[122, 63, 56], [132, 255, 255]], dtype=np.uint8),
        "orange": np.array([[100, 40, 170], [120, 255, 255]], dtype=np.uint8),
        "yellow": np.array([[80, 90, 100], [100, 255, 255]], dtype=np.uint8),
        "green": np.array([[50, 150, 60], [80, 210, 200]], dtype=np.uint8),
        "blue": np.array([[0, 70, 25], [40, 255, 210]], dtype=np.uint8),
    }

    color_dict = {
        "red": (0, 0, 255),  # 红色
        "orange": (0, 165, 255),  # 橘色
        "yellow": (0, 255, 255),  # 黄色
        "green": (0, 128, 0),  # 綠色
        "blue": (255, 0, 0),  # 藍色
        "white": (255, 255, 255),  # 白色
    }
    hsv_image = cv2.cvtColor(masked_image, cv2.COLOR_RGB2HSV)
    max_color = None
    pixel_counts = {}
    for color_name, hsv_range in hsv_ranges.items():
        lower_bound = hsv_range[0]
        upper_bound = hsv_range[1]

        mask = cv2.inRange(hsv_image, lower_bound, upper_bound)
        pixel_counts[color_name] = cv2.countNonZero(mask)

    max_color = max(pixel_counts, key=pixel_counts.get)
    return max_color, color_dict[max_color]


def correct_coordinates(approx_points_pack, epsilon):
    correct_approx_points_pack = []
    # 取出所有端點座標
    all_points = np.vstack([point for points in approx_points_pack for point in points])
    # 做兩次凝聚相鄰座標點
    all_points = merge_close_points(all_points, 3.5 * epsilon, iter=2)
    update_points = set(tuple(point) for point in all_points)
    # 更新舊多邊形端點座標成距離最近的凝聚座標點，並且紀錄有被更新到的座標點

    for approx_points in approx_points_pack:
        # new_points = np.empty((0, 1, 2), dtype=int)
        for i in range(len(approx_points)):
            target_point = np.array(approx_points[i])
            distances = np.linalg.norm(all_points - target_point, axis=1)
            nearest_index = np.argmin(distances)
            # if np.any(approx_points[i] != all_points[nearest_index]):
                # update_points.add(tuple(all_points[nearest_index]))
                # new_points = np.concatenate(
                #     (new_points, [[all_points[nearest_index]]]), axis=0
                # )
            approx_points[i] = all_points[nearest_index]

        # correct_approx_points_pack.append(new_points)
        correct_approx_points_pack.append(approx_points)
    # 將校正過的多邊形同梱包，與有被更新到的座標返回
    return correct_approx_points_pack, update_points


def merge_close_points(coordinates, threshold_distance, iter):
    # 建立KD樹

    merged_points = []
    # 進行最近鄰查詢，找到接近的點
    for i in range(iter):
        kdtree = cKDTree(coordinates)
        merged_points = []
        visited_points = set()
        for j, point in enumerate(coordinates):
            if j not in visited_points:
                # 找到與當前點接近的所有點
                close_indices = kdtree.query_ball_point(point, threshold_distance)
                visited_points.update(close_indices)

                # 計算接近的點的平均值，作為合併後的新點
                avg_x = sum(coordinates[idx][0] for idx in close_indices) / len(
                    close_indices
                )
                avg_y = sum(coordinates[idx][1] for idx in close_indices) / len(
                    close_indices
                )
                if i != 0 or len(close_indices) > 1:
                    merged_points.append((avg_x, avg_y))
        coordinates = merged_points
    return np.int0(merged_points)

**[Optional]**  
Make sure you have already installed OpenCV with CUDA support.  

Filters:


In [ ]:
sobelFilterX = cv2.cuda.createSobelFilter(0, cv2.CV_32F, 1, 0, 3)
sobelFilterY = cv2.cuda.createSobelFilter(0, cv2.CV_32F, 0, 1, 3)
cannyDetector = cv2.cuda.createCannyEdgeDetector(25, 100)



#### Main Function

##### CPU version

In [ ]:
# CPU_VERSION
# 物件檢測----------------------------------------------------
img = cv2.imread("test6.jpg")
height = 600
scale = height / img.shape[0]
img = cv2.resize(img, None, fx=scale, fy=scale)
results = model(img)[0]
img_r = results[0].plot(labels=False, boxes=False, masks=True)
mask = results.masks[0]
box = results.boxes[0]
# for box, mask in zip(results.boxes, results.masks):
segment = np.where(mask.data.cpu()[0] != 0, 255, 0).astype(np.uint8)
scale = height / segment.shape[0]
segment = cv2.resize(segment, None, fx=scale, fy=scale)
offset = (segment.shape[1] - img.shape[1]) // 2
segment = segment[:, offset : offset + img.shape[1]]
masked_image = cv2.bitwise_and(img, img, mask=segment)
x1, y1, x2, y2 = box.xyxyn.cpu()[0].numpy()
x1, x2 = map(lambda x: int(x * masked_image.shape[1]), [x1, x2])
y1, y2 = map(lambda x: int(x * masked_image.shape[0]), [y1, y2])
masked_image = masked_image[y1:y2, x1:x2]
box_scale = 300 / masked_image.shape[0]
masked_image = cv2.resize(masked_image, None, fx=box_scale, fy=box_scale)

# 顏色檢測----------------------------------------------------
color, color_rgb = color_detect(masked_image)

# 角點檢測----------------------------------------------------
cv2.convertScaleAbs(masked_image, masked_image, 1, 25)  # 畫面亮度調亮
cv2.detailEnhance(masked_image, masked_image, 10, 0.01)  # 強化細節
gray = cv2.cvtColor(masked_image, cv2.COLOR_BGR2GRAY)

# 高斯模糊減少噪點
kernel_size = 9
gray = cv2.GaussianBlur(gray, (kernel_size, kernel_size), 1)

cv2.edgePreservingFilter(gray, gray, 1, 120, 0.1)  # 減少噪點，而不模糊邊緣
# gray = cv2.bilateralFilter(gray, 40, 10, 20)
# sobel算子偵測邊緣
sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)  # 計算梯度幅值
sobel_image = np.sqrt(sobel_x**2 + sobel_y**2)  # 將梯度幅值轉換為8位無符號整數
sobel_image = np.uint8(255 * sobel_image / np.max(sobel_image))

cv2.edgePreservingFilter(sobel_image,sobel_image,1,120,0.1)
# 將遮罩輪廓繪製，避免邊緣偵測不清輪廓
segment = segment[y1:y2, x1:x2]
segment = cv2.resize(segment, None, fx=box_scale, fy=box_scale)
contours_outer, _ = cv2.findContours(
    segment, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE
)
contour_outer = max(contours_outer, key=cv2.contourArea)
sobel_image = cv2.drawContours(sobel_image, [contour_outer], -1, (255, 255, 255), 1)

# sobel_image = cv2.bilateralFilter(sobel_image, 5, 10, 20)

# 使用Canny算子邊緣檢測
canny_image = cv2.Canny(sobel_image, 0, 100)
kernel = np.ones((3, 3), dtype=np.uint8)
canny_image = cv2.dilate(canny_image, kernel, iterations=2)
canny_image = cv2.erode(canny_image, kernel, iterations=1)

# 找尋輪廓，並繪製
contours, hierarchy = cv2.findContours(
    canny_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE
)
contour_image = np.zeros(
    (masked_image.shape[0], masked_image.shape[1], 3), dtype=np.uint8
)
contour_image = cv2.drawContours(contour_image, contours, -1, (0, 0, 255), 1)
# 找出所有輪廓的近似多邊形
epsilon = 0.02 * cv2.arcLength(contour_outer, True)
outer_points = cv2.approxPolyDP(contour_outer, epsilon, True)
delta = 0.04 * cv2.arcLength(contour_outer, True)

# 把所有多邊形點打包整理，相鄰的端點合併，校正位置
approx_points_pack = []
isClosed = True
contourArea_outer = cv2.contourArea(contour_outer)
epsilon_outer = 0.01 * cv2.arcLength(contour_outer, isClosed)
for i, contour in enumerate(contours):
    if i == 0:
        contour = contour_outer
    if cv2.contourArea(contour) < 0.05 * contourArea_outer:
        continue
    epsilon = 0.02 * cv2.arcLength(contour, isClosed)
    approx_points = cv2.approxPolyDP(contour, epsilon, isClosed)
    approx_points_pack.append(approx_points)
approx_image = np.zeros(
    (masked_image.shape[0], masked_image.shape[1]), dtype=np.uint8
)
for approx_points in approx_points_pack:
    approx_image = cv2.polylines(approx_image, [approx_points], isClosed, (255), 2)
# 將打包的多邊形端點整理校正位置，首項為校正後的多邊形，次項為校正後的各個座標點
correct_approx_points_pack, update_points = correct_coordinates(
    approx_points_pack, epsilon=epsilon_outer
)
correct_approx_image = np.zeros(
    (masked_image.shape[0], masked_image.shape[1]), dtype=np.uint8
)
for correct_approx_points in correct_approx_points_pack:
    correct_approx_image = cv2.polylines(
        correct_approx_image, [correct_approx_points], isClosed, (255), 2
    )

for point in update_points:
    x, y = point
    cv2.circle(masked_image, (x, y), 10, color_rgb, -1)
    x = int(x / box_scale) + x1
    y = int(y / box_scale) + y1
    cv2.circle(img, (x, y), 10, color_rgb, -1)

# img = cv2.resize(img, None, fx=scale, fy=scale)
cv2.imshow("origin", img)
cv2.imshow("masked", masked_image)
cv2.imshow("gray", gray)
cv2.imshow("sobel", sobel_image)
cv2.imshow("canny", canny_image)
cv2.imshow("contour", contour_image)
cv2.imshow("approx", approx_image)
cv2.imshow("corner_masked", masked_image)
cv2.imshow("correct approx", correct_approx_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


##### GPU version

Make sure you have already installed OpenCV with CUDA support.

In [ ]:
#GPU_VERSION
# 物件檢測----------------------------------------------------
img = cv2.imread("test7.jpg")
height = 600
scale = height / img.shape[0]
img = cv2.resize(img, None, fx=scale, fy=scale)
results = model(img)[0]
img = results[0].plot(labels=False, boxes=False, masks=False)
mask = results.masks[3]
box = results.boxes[3]
segment = np.where(mask.data.cpu()[0] != 0, 255, 0).astype(np.uint8)
scale = height / segment.shape[0]
segment = cv2.resize(segment, None, fx=scale, fy=scale)
offset = (segment.shape[1] - img.shape[1]) // 2
segment = segment[:, offset : offset + img.shape[1]]
masked_image = cv2.bitwise_and(img, img, mask=segment)
x1, y1, x2, y2 = box.xyxyn.cpu()[0].numpy()
x1, x2 = map(lambda x: int(x * masked_image.shape[1]), [x1, x2])
y1, y2 = map(lambda x: int(x * masked_image.shape[0]), [y1, y2])
masked_image = masked_image[y1:y2, x1:x2]
box_scale = 300 / masked_image.shape[0]
masked_image = cv2.resize(masked_image, None, fx=box_scale, fy=box_scale)

# 顏色檢測----------------------------------------------------
color, color_rgb = color_detect(masked_image)  

# 角點檢測----------------------------------------------------
gpu_img = cv2.cuda.GpuMat()

cv2.convertScaleAbs(masked_image, masked_image, 1, 25)  # 畫面亮度調亮
gray = cv2.cvtColor(masked_image, cv2.COLOR_BGR2GRAY)
# 高斯模糊 減少噪點
kernel_size = 9
gray = cv2.GaussianBlur(gray, (kernel_size, kernel_size), 1)

# 雙邊濾波 減少噪點，而不模糊邊緣
gpu_img.upload(gray)
gpu_img = cv2.cuda.bilateralFilter(gpu_img, 40, 10, 20)
# sobel算子偵測邊緣
sobel_x = sobelFilterX.apply(gpu_img).download()
sobel_y = sobelFilterY.apply(gpu_img).download()
sobel_image = np.sqrt(sobel_x**2 + sobel_y**2)  # 將梯度幅值轉換為8位無符號整數
sobel_image = np.uint8(255 * sobel_image / np.max(sobel_image))

# 將遮罩輪廓繪製，避免邊緣偵測不清輪廓
segment = segment[y1:y2, x1:x2]
segment = cv2.resize(segment, None, fx=box_scale, fy=box_scale)
contours_outer, _ = cv2.findContours(segment, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contour_outer = max(contours_outer, key=cv2.contourArea)
sobel_image = cv2.drawContours(sobel_image, [contour_outer], -1, (255, 255, 255), 1)

gpu_img.upload(sobel_image)
gpu_img = cv2.cuda.bilateralFilter(gpu_img, 5, 10, 20)

# 使用Canny算子邊緣檢測
cannyDetector.detect(gpu_img,gpu_img)
canny_edges=gpu_img.download()
canny_image = cv2.Canny(sobel_image, 0, 100)
kernel = np.ones((3, 3), dtype=np.uint8)
canny_image = cv2.dilate(canny_image, kernel, iterations=2)
canny_image = cv2.erode(canny_image, kernel, iterations=1)

# 找尋輪廓，並繪製
contours, hierarchy = cv2.findContours(
    canny_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE
)
contour_image = np.zeros(
    (masked_image.shape[0], masked_image.shape[1], 3), dtype=np.uint8
)
contour_image = cv2.drawContours(contour_image, contours, -1, (0, 0, 255), 1)
# 找出所有輪廓的近似多邊形
epsilon = 0.02 * cv2.arcLength(contour_outer, True)
outer_points = cv2.approxPolyDP(contour_outer, epsilon, True)
delta = 0.04 * cv2.arcLength(contour_outer, True)


# 把所有多邊形點打包整理，相鄰的端點合併，校正位置
approx_points_pack = []
isClosed = True
contourArea_outer = cv2.contourArea(contour_outer)
epsilon_outer = 0.01 * cv2.arcLength(contour_outer, isClosed)
for i, contour in enumerate(contours):
    if i == 0:
        contour = contour_outer
    if cv2.contourArea(contour) < 0.05 * contourArea_outer:
        continue
    epsilon = 0.02 * cv2.arcLength(contour, isClosed)
    approx_points = cv2.approxPolyDP(contour, epsilon, isClosed)
    approx_points_pack.append(approx_points)
approx_image = np.zeros((masked_image.shape[0], masked_image.shape[1]), dtype=np.uint8)
for approx_points in approx_points_pack:
    approx_image = cv2.polylines(approx_image, [approx_points], isClosed, (255), 2)
# 將打包的多邊形端點整理校正位置，首項為校正後的多邊形，次項為校正後的各個座標點
correct_approx_points_pack, update_points = correct_coordinates(
    approx_points_pack, epsilon=epsilon_outer
)
correct_approx_image = np.zeros(
    (masked_image.shape[0], masked_image.shape[1]), dtype=np.uint8
)
for correct_approx_points in correct_approx_points_pack:
    correct_approx_image = cv2.polylines(
        correct_approx_image, [correct_approx_points], isClosed, (255), 2
    )


for point in update_points:
    x, y = point
    cv2.circle(masked_image, (x, y), 5, color_rgb, -1)
    x = int(x / box_scale) + x1
    y = int(y / box_scale) + y1
    cv2.circle(img, (x, y), 5, color_rgb, -1)
img = cv2.resize(img, None, fx=scale, fy=scale)
cv2.imshow("origin", img)
cv2.imshow("masked", masked_image)
cv2.imshow("gray", gray)
cv2.imshow("sobel", sobel_image)
cv2.imshow("canny", canny_image)
cv2.imshow("contour", contour_image)
cv2.imshow("approx", approx_image)
cv2.imshow("corner_masked", masked_image)
cv2.imshow("correct approx", correct_approx_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

### **Real time object detection**

Open web_cam 

In [ ]:
cap = cv2.VideoCapture(0)

Close web_cam (Run this after finish the task)

In [ ]:
cap.release()

#### Functions and Filters define

Functions:

In [ ]:
def color_detect(masked_image):
    hsv_ranges = {
        "white": np.array([[0, 15, 200], [75, 70, 255]], dtype=np.uint8),
        "red": np.array([[122, 63, 56], [132, 255, 255]], dtype=np.uint8),
        "orange": np.array([[100, 40, 170], [120, 255, 255]], dtype=np.uint8),
        "yellow": np.array([[80, 90, 100], [100, 255, 255]], dtype=np.uint8),
        "green": np.array([[50, 150, 60], [80, 210, 200]], dtype=np.uint8),
        "blue": np.array([[0, 70, 25], [40, 255, 210]], dtype=np.uint8),
    }

    color_dict = {
        "red": (0, 0, 255),  # 红色
        "orange": (0, 165, 255),  # 橘色
        "yellow": (0, 255, 255),  # 黄色
        "green": (0, 128, 0),  # 绿色
        "blue": (255, 0, 0),  # 蓝色
        "white": (255, 255, 255),  # 白色
    }
    hsv_image = cv2.cvtColor(masked_image, cv2.COLOR_RGB2HSV)
    max_color = None
    pixel_counts = {}
    for color_name, hsv_range in hsv_ranges.items():
        lower_bound = hsv_range[0]
        upper_bound = hsv_range[1]

        mask = cv2.inRange(hsv_image, lower_bound, upper_bound)
        pixel_counts[color_name] = cv2.countNonZero(mask)

    print(pixel_counts.get)
    max_color = max(pixel_counts, key=pixel_counts.get)
    return max_color, color_dict[max_color]



def correct_coordinates(approx_points_pack, epsilon):
    correct_approx_points_pack = []
    update_points = set()
    # 取出所有端點座標
    all_points = np.vstack([point for points in approx_points_pack for point in points]) 
    # 做兩次凝聚相鄰座標點
    all_points = merge_close_points(all_points, 3.5 * epsilon)
    all_points = merge_close_points(all_points, 3.5 * epsilon)

    # 更新舊多邊形端點座標成距離最近的凝聚座標點，並且紀錄有被更新到的座標點
    for approx_points in approx_points_pack:
        for i in range(len(approx_points)):
            target_point = np.array(approx_points[i])
            distances = np.linalg.norm(all_points - target_point, axis=1)
            nearest_index = np.argmin(distances)
            if np.any(approx_points[i] != all_points[nearest_index]):
                update_points.add(tuple(all_points[nearest_index]))
            approx_points[i] = all_points[nearest_index]
        correct_approx_points_pack.append(approx_points)
    # 將校正過的多邊形同梱包，與有被更新到的座標返回
    return correct_approx_points_pack, update_points

def merge_close_points(coordinates, threshold_distance):
    # 建立KD樹
    kdtree = cKDTree(coordinates)

    # 進行最近鄰查詢，找到接近的點
    merged_points = []
    visited_points = set()
    for i, point in enumerate(coordinates):
        if i not in visited_points:
            # 找到與當前點接近的所有點
            close_indices = kdtree.query_ball_point(point, threshold_distance)
            visited_points.update(close_indices)

            # 計算接近的點的平均值，作為合併後的新點
            avg_x = sum(coordinates[idx][0] for idx in close_indices) / len(
                close_indices
            )
            avg_y = sum(coordinates[idx][1] for idx in close_indices) / len(
                close_indices
            )
            merged_points.append((avg_x, avg_y))

    return np.int0(merged_points)


**[Optional]**  
Make sure you have already installed OpenCV with CUDA support.  

Filters:


In [ ]:
sobelFilterX = cv2.cuda.createSobelFilter(0, cv2.CV_32F, 1, 0, 3)
sobelFilterY = cv2.cuda.createSobelFilter(0, cv2.CV_32F, 0, 1, 3)
cannyDetector = cv2.cuda.createCannyEdgeDetector(25, 100)


#### CPU version

In [ ]:
# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        start_total = time.time()
        height = 600
        scale = height / frame.shape[0]
        frame = cv2.resize(
            frame, None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR
        )

        # Run YOLOv8 inference on the frame
        results = model(frame)
        # img = results[0].plot(labels=False, boxes=False, masks=False)
        img = results[0].plot(labels=False, boxes=False, masks=True)
        colors = []
        gpu_img = cv2.cuda_GpuMat()
        masks = results[0].masks
        # boxes = results[0].boxes
        if masks is None:
            print("No Mask")

            cv2.imshow("YOLOv8 Inference", img)

            continue
        for mask in masks:
            segment = np.where(mask.data.cpu()[0] != 0, 255, 0).astype(np.uint8)
            segment = cv2.resize(
                segment, None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR
            )
            offset = (segment.shape[1] - img.shape[1]) // 2
            segment = segment[:, offset : offset + img.shape[1]]
            masked_image = cv2.bitwise_and(img, img, mask=segment)
            indices = np.where(segment != 0)
            y1, y2 = min(indices[0]) - 5, max(indices[0]) + 5
            x1, x2 = min(indices[1]) - 5, max(indices[1]) + 5
            masked_image = masked_image[y1:y2, x1:x2]

            if not ( masked_image.shape[0] > 0 and masked_image.shape[1] > 0 and masked_image is not None):
                print("box not found")
                continue
            # x1, y1, x2, y2 = box.xyxyn.cpu()[0].numpy()
            # x1, x2 = map(lambda x: int(x * masked_image.shape[1]), [x1, x2])
            # y1, y2 = map(lambda x: int(x * masked_image.shape[0]), [y1, y2])
            # masked_image = masked_image[y1:y2, x1:x2]
            box_scale = 300 / masked_image.shape[0]
            masked_image = cv2.resize(masked_image, None, fx=box_scale, fy=box_scale)
            start = time.time()
            color, color_rgb = color_detect(masked_image)
            end = time.time()

            colors.append(color)

            cv2.convertScaleAbs(masked_image, masked_image, 1, 25)
            cv2.detailEnhance(masked_image, masked_image, 10, 0.1)

            gray = cv2.cvtColor(masked_image, cv2.COLOR_BGR2GRAY)

            kernel_size = 21
            gray = cv2.GaussianBlur(gray, (kernel_size, kernel_size), 1)

            cv2.edgePreservingFilter(gray, gray, 1, 120, 0.1)

            sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
            sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)

            # 計算梯度幅值
            gradient_magnitude = np.sqrt(sobel_x**2 + sobel_y**2)

            # 將梯度幅值轉換為8位無符號整數
            gradient_magnitude = np.uint8(
                255 * gradient_magnitude / np.max(gradient_magnitude)
            )
            # 使用Canny邊緣檢測
            segment = segment[y1:y2, x1:x2]
            segment = cv2.resize(segment, None, fx=box_scale, fy=box_scale)
            contours_outer, _ = cv2.findContours(
                segment, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE
            )
            contour_outer = max(contours_outer, key=cv2.contourArea)
            gradient_magnitude = cv2.drawContours(
                gradient_magnitude, [contour_outer], -1, (255, 255, 255), 1
            )

            cv2.edgePreservingFilter(
                gradient_magnitude, gradient_magnitude, 1, 120, 0.1
            )
            canny_edges = cv2.Canny(gradient_magnitude, 25, 100)

            kernel = np.ones((3, 3), dtype=np.uint8)
            canny_edges = cv2.dilate(canny_edges, kernel, iterations=2)

            erode = cv2.erode(canny_edges, kernel, iterations=1)
            contours, hierarchy = cv2.findContours(
                erode, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE
            )
            contour_image = np.zeros(
                (masked_image.shape[0], masked_image.shape[1], 3), dtype=np.uint8
            )
            contour_image = cv2.drawContours(
                contour_image, contours, -1, (0, 0, 255), 1
            )

            epsilon = 0.02 * cv2.arcLength(contour_outer, True)
            outer_points = cv2.approxPolyDP(contour_outer, epsilon, True)
            delta = 0.04 * cv2.arcLength(contour_outer, True)

            # approx_image = np.zeros((masked_image.shape[0], masked_image.shape[1]), dtype=np.uint8)
            # approx_image_correct = np.zeros(
            #     (masked_image.shape[0], masked_image.shape[1]), dtype=np.uint8
            # )

            approx_points_pack = []
            isClosed = True

            contourArea_outer = cv2.contourArea(contour_outer)
            epsilon_outer = 0.01 * cv2.arcLength(contour_outer, isClosed)

            for i, contour in enumerate(contours):
                if i == 0:
                    contour = contour_outer
                if cv2.contourArea(contour) < 0.05 * contourArea_outer:
                    continue
                epsilon = 0.02 * cv2.arcLength(contour, isClosed)
                approx_points = cv2.approxPolyDP(contour, epsilon, isClosed)
                approx_points_pack.append(approx_points)

            new_approx_points_pack, update_points = correct_coordinates(
                approx_points_pack, epsilon=epsilon_outer
            )

            for point in update_points:
                x, y = point
                x = int(x / box_scale) + x1
                y = int(y / box_scale) + y1

                cv2.circle(img, (x, y), 5, color_rgb, -1)  # -1 表示填滿圓圈

            # print("執行時間：%f 秒" % (end - start))
        end_total = time.time()
        print("總執行時間：%f 秒" % (end_total - start_total), "####################")
        # img = cv2.resize(img, None, fx=scale, fy=scale)
        frame_number = int(1 / (end_total - start_total))
        text = f"[CPU] FPS: {frame_number}"
        cv2.putText(img, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2)

        cv2.imshow("YOLOv8 Inference", img)
        # cv2.imshow("YOLOv8 masked", masked_image)
        # cv2.imshow("YOLOv8 contour", contour_image)
        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window


cv2.destroyAllWindows()

#### GPU version

Make sure you have already installed OpenCV with CUDA support.

In [27]:
# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        start_total = time.time()
        height = 600
        scale = height / frame.shape[0]
        frame = cv2.resize(
            frame, None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR
        )

        # Run YOLOv8 inference on the frame
        results = model(frame)
        img = results[0].plot(labels=False, boxes=False, masks=False)
        img_masked = results[0].plot(labels=False, boxes=False, masks=True)
        colors = []
        gpu_img = cv2.cuda_GpuMat()
        masks = results[0].masks
        boxes = results[0].boxes
        if masks is None:
            print("No Mask")

            cv2.imshow("YOLOv8 Inference", img)

            continue
        for mask, box in zip(masks, boxes):
            segment = np.where(mask.data.cpu()[0] != 0, 255, 0).astype(np.uint8)
            segment = cv2.resize(segment, None, fx=scale, fy=scale)
            offset = (segment.shape[1] - img.shape[1]) // 2
            segment = segment[:, offset : offset + img.shape[1]]
            masked_image = cv2.bitwise_and(img, img, mask=segment)
            x1, y1, x2, y2 = box.xyxyn.cpu()[0].numpy()
            x1, x2 = map(lambda x: int(x * masked_image.shape[1]), [x1, x2])
            y1, y2 = map(lambda x: int(x * masked_image.shape[0]), [y1, y2])
            masked_image = masked_image[y1:y2, x1:x2]
            if not ( masked_image.shape[0] > 0 and masked_image.shape[1] > 0 and masked_image is not None):
                print("box not found")
                continue
            box_scale = 300 / masked_image.shape[0]
            masked_image = cv2.resize(masked_image, None, fx=box_scale, fy=box_scale)
            start = time.time()
            color, color_rgb = color_detect(masked_image)
            end = time.time()

            colors.append(color)

            cv2.convertScaleAbs(masked_image, masked_image, 1, 25)

            gray = cv2.cvtColor(masked_image, cv2.COLOR_BGR2GRAY)

            kernel_size = 21
            gray = cv2.GaussianBlur(gray, (kernel_size, kernel_size), 1)
            gpu_img.upload(gray)
            # cv2.cuda.fastNlMeansDenoising(gpu_img,10,gpu_img,7,21)
            gpu_img = cv2.cuda.bilateralFilter(gpu_img, 40, 10, 20)
            gray = gpu_img.download()

            gpu_img.upload(gray)
            # 雙邊濾波
            sobel_x = sobelFilterX.apply(gpu_img).download()
            sobel_y = sobelFilterY.apply(gpu_img).download()

            # 計算梯度幅值
            gradient_magnitude = np.sqrt(sobel_x**2 + sobel_y**2)

            # 將梯度幅值轉換為8位無符號整數
            gradient_magnitude = np.uint8(
                255 * gradient_magnitude / np.max(gradient_magnitude)
            )
        
            segment = segment[y1:y2, x1:x2]
            segment = cv2.resize(segment, None, fx=box_scale, fy=box_scale)
            contours_outer, _ = cv2.findContours(
                segment, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE
            )
            contour_outer = max(contours_outer, key=cv2.contourArea)
            gradient_magnitude = cv2.drawContours(
                gradient_magnitude, [contour_outer], -1, (255, 255, 255), 2
            )

            gpu_img.upload(gradient_magnitude)
            gpu_img = cv2.cuda.bilateralFilter(gpu_img, 5, 10, 20)
            gradient_magnitude = gpu_img.download()

            canny_edges = cv2.Canny(gradient_magnitude, 25, 100)

            kernel = np.ones((3, 3), dtype=np.uint8)
            canny_edges = cv2.dilate(canny_edges, kernel, iterations=2)

            erode = cv2.erode(canny_edges, kernel, iterations=1)
            contours, hierarchy = cv2.findContours(
                erode, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE
            )
            contour_image = np.zeros(
                (masked_image.shape[0], masked_image.shape[1], 3), dtype=np.uint8
            )
            contour_image = cv2.drawContours(
                contour_image, contours, -1, (0, 0, 255), 1
            )

            epsilon = 0.02 * cv2.arcLength(contour_outer, True)
            outer_points = cv2.approxPolyDP(contour_outer, epsilon, True)
            delta = 0.04 * cv2.arcLength(contour_outer, True)

            approx_points_pack = []
            isClosed = True

            contourArea_outer = cv2.contourArea(contour_outer)
            epsilon_outer = 0.01 * cv2.arcLength(contour_outer, isClosed)

            for i, contour in enumerate(contours):
                if i == 0:
                    contour = contour_outer
                if cv2.contourArea(contour) < 0.05 * contourArea_outer:
                    continue
                epsilon = 0.02 * cv2.arcLength(contour, isClosed)
                approx_points = cv2.approxPolyDP(contour, epsilon, isClosed)
                approx_points_pack.append(approx_points)
            if len(approx_points_pack)==0:
                continue
            new_approx_points_pack, update_points = correct_coordinates(
                approx_points_pack, epsilon=epsilon_outer
            )

            for point in update_points:
                x, y = point
                x = int(x / box_scale) + x1
                y = int(y / box_scale) + y1

                cv2.circle(img, (x, y), 5, color_rgb, -1)  # -1 表示填滿圓圈

            # print("執行時間：%f 秒" % (end - start))
        end_total = time.time()
        print("總執行時間：%f 秒" % (end_total - start_total), "####################")
        img = cv2.resize(img, None, fx=scale, fy=scale)
        frame_number = int(1 / (end_total - start_total))
        text = f"[GPU] FPS: {frame_number}"
        cv2.putText(img, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        cv2.imshow("Inference", img)
        cv2.imshow("masked", masked_image)
        cv2.imshow("contour", contour_image)
        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

cv2.destroyAllWindows()



0: 480x640 3 cubes, 19.0ms
Speed: 4.0ms preprocess, 19.0ms inference, 4.2ms postprocess per image at shape (1, 3, 480, 640)


C:\Users\user\AppData\Local\Temp\ipykernel_27020\3288770374.py:86: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  return np.int0(merged_points)

0: 480x640 3 cubes, 21.4ms
Speed: 3.0ms preprocess, 21.4ms inference, 5.0ms postprocess per image at shape (1, 3, 480, 640)


總執行時間：0.111843 秒 ####################



0: 480x640 3 cubes, 23.5ms
Speed: 4.0ms preprocess, 23.5ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)


總執行時間：0.120533 秒 ####################
總執行時間：0.146535 秒 ####################


...

總執行時間：0.059024 秒 ####################
總執行時間：0.055019 秒 ####################
總執行時間：0.053024 秒 ####################



0: 480x640 2 cubes, 8.0ms
Speed: 2.0ms preprocess, 8.0ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 cubes, 10.0ms
Speed: 2.0ms preprocess, 10.0ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)


總執行時間：0.058002 秒 ####################
總執行時間：0.060998 秒 ####################
總執行時間：0.055020 秒 ####################



0: 480x640 2 cubes, 10.0ms
Speed: 2.0ms preprocess, 10.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)


總執行時間：0.066999 秒 ####################
